## Algoritmia
### Práctica 5
El objetivo de esta práctica es implementar algoritmos voraces: mochila y Prim.

Se pide la implementación de las clases y/o funciones que aparecen a continuación.

Las instrucción "pass" que aparecen en el cuerpo de las clases o funciones, se debe sustituir por la implementación adecuada.

Para cada clase o función que se pide se proporciona una o más funciones con algunos tests.

Al llamar a las funciones de test no debería saltar ninguna aserción.

### Problema de la mochila
#### Clase `ObjetoMochila`

In [11]:
class ObjetoMochila:
    """
    Clase que representa un objeto caracterizado por su peso y valor.
    Tiene definidos los siguientes operadores: ==, !=, <, <=, >, >=, +, +=, 
        *, *=, str() y repr().
    El orden en las comparaciones viene dado por valor/peso.
    La multiplicación está definida con números, no con otros objetos.
    El formato de las cadenas debe ser "(peso, valor)"
    """

    def __init__(self, peso, valor):
        self.pesoM=peso
        self.valorM=valor

    def peso(self):
        return self.pesoM

    def valor(self):
        return self.valorM

    def valor_por_peso(self):
        """El valor por unidad de peso"""
        return self.valorM/self.pesoM
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            if self.pesoM==other.pesoM and self.valorM==other.valorM:
                return True
            return False
        return False
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __lt__(self, other):
        return self.valor_por_peso() < other.valor_por_peso()
    
    def __le__(self, other):
        return self.valor_por_peso() <= other.valor_por_peso()
    
    def __gt__(self, other):
        return self.valor_por_peso() > other.valor_por_peso()
    
    def __ge__(self, other):
        return self.valor_por_peso() >= other.valor_por_peso()
    
    def __add__(self, other):
        sumaPeso=self.pesoM + other.pesoM
        sumaValor=self.valorM + other.valorM
        return ObjetoMochila(sumaPeso,sumaValor)
    
    def __iadd__(self, other):
        self = self + other
        return self
    
    def __mul__(self, other):
        mulPeso=self.pesoM * other
        mulValor=self.valorM * other
        return ObjetoMochila(mulPeso,mulValor)
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __imul__(self, other):
        self = self * other
        return self
    
    def __repr__(self):
        return str((self.pesoM, self.valorM))
    
    def __str__(self):
        return str((self.pesoM, self.valorM))

In [12]:
def test_objeto_mochila():
    """Tests para la clase ObjetoMochila."""

    o1 = ObjetoMochila(10, 20)
    assert o1.peso() == 10
    assert o1.valor() == 20
    assert o1.valor_por_peso() == 2

    o2 = ObjetoMochila(20, 10)        
    assert o2.peso() == 20
    assert o2.valor() == 10
    assert o2.valor_por_peso() == 0.5

    assert o1 == ObjetoMochila(10, 20)
    assert o2 == ObjetoMochila(20, 10)
    assert not (o1 == o2)
    assert not (o1 == 10)
    assert not (o1 == ObjetoMochila(1, 2))

    assert not (o1 != ObjetoMochila(10, 20))
    assert not (o2 != ObjetoMochila(20, 10))
    assert o1 != o2
    assert o1 != 10
    assert o1 != ObjetoMochila(1, 2)

    assert o1 > o2
    assert o1 >= o2
    assert not (o1 < o2)
    assert not (o1 <= o2)

    assert not (o2 > o1)
    assert not (o2 >= o1 )     
    assert o2 < o1
    assert o2 <= o1

    assert not (o1 > o1)
    assert o1 >= o1
    assert not (o1 < o1)
    assert o1 <= o1

    assert str(o1) == "(10, 20)"
    assert str(o2) == "(20, 10)"

    assert str([o1, o2])  == "[(10, 20), (20, 10)]"

    assert o1 + o2 == ObjetoMochila(30, 30)
    assert o1 + o1 == ObjetoMochila(20, 40)

    o1 += o1
    assert o1 == ObjetoMochila(20, 40)
    o1 += o2
    assert o1 == ObjetoMochila(40, 50)

    assert o1 * 0.1 == ObjetoMochila(4, 5)
    assert 0.2 * o1 == ObjetoMochila(8, 10)

    o1 *= 0.5
    assert o1 == ObjetoMochila(20, 25)
    o1 *= 0.2
    assert o1 == ObjetoMochila(4, 5)        

        
if __name__ == "__main__": 
    test_objeto_mochila()
    print("OK")  

OK


#### Clase `Mochila`

In [13]:
class Mochila:
    """
    Clase que representa una mochila que tiene una capacidad (en peso) y
    sobre la que se pueden insertar objetos.
    """

    def __init__(self, capacidad):
        self._capacidad = capacidad
        self._objetos = []

    def capacidad(self):
        return self._capacidad

    def peso(self):
        """peso total de los objetos en la mochila."""
        pesoTotal = 0
        for i in self._objetos:
            pesoTotal += i.peso()
        return pesoTotal

    def valor(self):
        """valor total de los objetos en la mochila."""
        valorTotal = 0
        for i in self._objetos:
            valorTotal += i.valor()
        return valorTotal

    def __len__(self):
        """Operador len(), indica el número de objetos en la mochila"""
        return len(self._objetos)

    def __contains__(self, objeto):
        """Operador 'in'"""
        if objeto in self._objetos:
            return True
        return False

    def __getitem__(self, k):
        """ 
        Acceso al elemento k-ésimo mediante "mochila[k]". 
        El orden de los elementos es el de inserción.
        """
        return self._objetos[k]

    def __delitem__(self, k):
        """ Eliminación de elementos mediante "del mochila[k]"."""
        del self._objetos[k]

    def __iadd__(self, objeto):
        """Operador +=, con instancias de ObjetoMochila.
        Si la mochila está llena no se añade.
        Si no cabe entero se inserta la fracción de objeto que quepa.
        """
        pesocabe = 0
        valor = 0
        
        if self.peso() < self._capacidad:
            if (self.peso() + objeto.peso()) <= self._capacidad:
                self._objetos.append(objeto)
            else:
                pesocabe = self._capacidad - self.peso()
                valor = (pesocabe / objeto.peso()) * objeto.valor()
                #valor = pesocabe * objeto.valor_por_peso()
                self._objetos.append(ObjetoMochila(pesocabe, valor))
        
        return self
    
    def __iter__(self):
        return iter(self._objetos)

In [14]:
def test_mochila():
    """Tests para la clase Mochila."""
    
    m = Mochila(100)
    assert m.capacidad() == 100
    assert m.peso() == 0
    assert m.valor() == 0
    assert len(m) == 0

    m += ObjetoMochila(10, 20)
    assert len(m) == 1
    assert m.capacidad() == 100
    assert m.peso() == 10
    assert m.valor() == 20

    m += ObjetoMochila(5, 15)
    assert len(m) == 2
    assert m.capacidad() == 100
    assert m.peso() == 15
    assert m.valor() == 35

    assert ObjetoMochila(10, 20) in m
    assert ObjetoMochila(20, 10) not in m
    assert ObjetoMochila(5, 15) in m
    assert ObjetoMochila(15, 5) not in m
    
    assert m[0] == ObjetoMochila(10, 20)
    assert m[1] == ObjetoMochila(5, 15)

    del m[0]
    assert len(m) == 1
    assert m.capacidad() == 100
    assert m.peso() == 5
    assert m.valor() == 15

    del m[0]
    assert len(m) == 0
    assert m.capacidad() == 100
    assert m.peso() == 0
    assert m.valor() == 0

    m += ObjetoMochila(50, 10)
    assert len(m) == 1
    assert m.peso() == 50
    assert m.valor() == 10
    m += ObjetoMochila(100, 50)  # solo cabe la mitad del objeto
    assert len(m) == 2
    assert m.peso() == 100
    assert m.valor() == 35
    m += ObjetoMochila(1, 5) # no cabe, no se inserta
    assert len(m) == 2
    assert m.peso() == 100
    assert m.valor() == 35

    
if __name__ == "__main__": 
    test_mochila()
    print("OK")  

OK


#### Algoritmo voraz para el problema de la mochila

In [15]:
def algoritmo_mochila_voraz(pesos, valores, capacidad):
    """
    Implementación del método voraz para el problema de la mochila.
    Devuelve el peso y valor de los objetos insertados.
    """
    mochila = Mochila(capacidad)
    valorPeso = []
    ValorPesoOrdenado = []

    for i in list(range(len(pesos))):
        valorPeso.append(valores[i]/pesos[i])

    valorPesoOrdenado = sorted(valorPeso, reverse=True)

    for i in valorPesoOrdenado:
        mochila += ObjetoMochila(pesos[valorPeso.index(i)],valores[valorPeso.index(i)])

    pesoTotal = mochila.peso()
    valorTotal = mochila.valor()
    
    return pesoTotal,valorTotal


In [16]:
def test_algoritmo_mochila_voraz():
    """Tests para la función algoritmo_mochila_voraz."""

    peso, valor = algoritmo_mochila_voraz( [10, 20, 30, 40, 50],
                                           [20, 30, 66, 40, 60], 100)
    assert peso == 100
    assert valor == 164

    # caso en el que caben todos los objetos
    peso, valor = algoritmo_mochila_voraz( [10, 20, 30, 40, 50],
                                           [20, 30, 66, 40, 60], 1000)
    assert peso == 150
    assert valor == 216

    # caso en el que solo se introduce un fragmento de objeto
    peso, valor = algoritmo_mochila_voraz( [10, 200, 30, 40, 50],
                                           [20, 900, 66, 40, 60], 100)
    assert peso == 100
    assert valor == 450
    
if __name__ == "__main__": 
    test_algoritmo_mochila_voraz()
    print("OK")     

OK


### Algoritmo de Prim
#### Implementaciones de Grafos

In [17]:
"""
Se usarán las implementaciones de grafos desarrolladas en la práctica 2.
Dichas implementaciones deben estar en un fichero denominado "grafos.py" que 
también hay que entregar.
"""

from grafos import GrafoMatriz, GrafoListas

gm = GrafoMatriz()
gl = GrafoListas()

#### Implementación del algoritmo

In [18]:
def arbol_extendido_prim(grafo):
    """
    Dado un grafo, devuelve otro con el árbol extendido obtenido mediante
    el algoritmo de Prim.
    """
    listaN=[]
    grafo_nuevo = GrafoMatriz()
    distancia=-1
    visitados=[]
    listavecinos=[]
    cont=0
    for nodos in grafo:
        listaN.append(nodos)        
    visitados.append(listaN[0])
    while sorted(listaN) != sorted(visitados):
        for nodo in visitados:
            for caca in grafo.vecinos(nodo):
                h = [nodo] + [caca[0],caca[1]]
                listavecinos.append(h)
        
        menor = None

        for x in listavecinos:

            o,a,peso = x
            if not a in visitados:
                if distancia==-1:
                    distancia=peso
                    menor = x
                else:
                    if distancia>peso:
                        distancia=peso
                        menor = x
                #cont=cont+1    
        o,a,distancia = menor
        grafo_nuevo.inserta(o, a, distancia)
        visitados.append(a)
        distancia = -1
    return grafo_nuevo
    


In [19]:
def test_arbol_extendido_prim():
    """Tests para la función arbol_extendido_prim"""
    
    for claseGrafo in (GrafoMatriz, GrafoListas):
    
        g = claseGrafo()

        for (origen, destino, peso) in (   
            ("c", "d", 5),
            ("e", "f", 2),
            ('a', 'b', 13),
            ('a', 'c', 8),
            ('a', 'd', 1),
            ('b', 'c', 15),
            ('c', 'e', 3),
            ('d', 'e', 4),
            ('d', 'f', 5)
        ):
            g.inserta(origen, destino, peso)
        assert len(g) == 6
        assert g.num_arcos() == 9

        a = arbol_extendido_prim(g)

        assert len(a) == 6
        assert a.num_arcos() == 5

        arcos = [("a", "b", 13), ("a", "d", 1), ("c", "e", 3), ("d", "e", 4),
                 ("e", "f", 2)]

        for origen, destino, peso in arcos:
            assert a[origen, destino] == peso
            assert a[destino, origen] == peso

        for origen in a:
            for destino, etiqueta in a.vecinos(origen):
                assert ((origen, destino, etiqueta) in arcos
                        or (destino, origen, etiqueta) in arcos)
            
if __name__ == "__main__": 
    test_arbol_extendido_prim()
    print("OK")             

OK
